In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence, pack_sequence
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import argparse
# import summary
from model_model import WIDAEV1
from model_dataset import DS_Tools
from model_dataset import WordDatasetWord as TrainDataset
from model_dataset import TargetVowelDataset as TestDataset
from model_dataset import Normalizer, DeNormalizer, TokenMap
from model_dataset import MelSpecTransformDB as TheTransform
from paths import *
from misc_my_utils import *
from misc_recorder import *
from model_loss import *
from model_padding import generate_mask_from_lengths_mat

In [2]:
train_guide_path = os.path.join(src_, "guide_train.csv")
valid_guide_path = os.path.join(src_, "guide_validation.csv")
test_guide_path = os.path.join(src_, "guide_test.csv")

In [3]:
train_guide = pd.read_csv(train_guide_path)
valid_guide = pd.read_csv(valid_guide_path)
test_guide = pd.read_csv(test_guide_path)

concatenated_guide = pd.concat([train_guide, valid_guide, test_guide])

unique_words = concatenated_guide['word'].unique()
print(unique_words)


[nan 'chapter' 'ninety' ... 'sceptics' 'commingled' 'slided']


In [6]:
# nan should be those silent parts. We still include a nan in the list, but we will not use it.

nan_rows = concatenated_guide[concatenated_guide['word'].isna()]
print(nan_rows)


       segment              file   id  startTime  endTime  nSample  word_id  \
0          sil  1034-121119-0000    0       0.00     0.19     3040      NaN   
14          sp  1034-121119-0000   14       1.72     2.45    11680      NaN   
19          sp  1034-121119-0000   19       3.16     6.05    46240      NaN   
37          sp  1034-121119-0000   37       7.71     7.86     2400      NaN   
38         sil  1034-121119-0001    0       0.00     0.32     5120      NaN   
...        ...               ...  ...        ...      ...      ...      ...   
376735      sp   887-123291-0042   48       4.17     4.56     6240      NaN   
376754      sp   887-123291-0042   67       6.65     7.31    10560      NaN   
376780      sp   887-123291-0042   93       9.18     9.67     7840      NaN   
376783      sp   887-123291-0042   96      10.18    10.30     1920      NaN   
376840      sp   887-123291-0042  153      15.10    15.24     2240      NaN   

       word  in_id segment_nostress stress_type  \


In [7]:
import pickle
unique_words_list = unique_words.tolist()

# Specify the file path where you want to save the list
file_path = os.path.join(src_, "unique_words_list.dict")

# Save the list using pickle
with open(file_path, "wb") as file:
    pickle.dump(unique_words_list, file)


In [10]:
word2idx = {word: idx for idx, word in enumerate(unique_words_list)}

In [13]:
word2idx[np.nan]

0

In [ ]:
import pickle